Other languages with
gradual type systems are Microsoft’s TypeScript, Dart (the language of the
Flutter SDK, created by Google), and Hack (a dialect of PHP supported by
Facebook’s HHVM virtual machine).